# Generador de personas aleatorias

In [59]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import re
import pytz
import unicodedata
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder

In [82]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv')
ciudades = pd.read_csv('./catalogos/ciudades.csv')

def quitar_acentos(s):
	return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def generate_random_data(lat, lon, num_rows = 5, save_file_name = None):
	data = []
	for _ in range(num_rows):
		hex1 = '%012x' % rm.randrange(16 ** 12)
		flt = float(rm.randint(0, 100))
		dec_lat = rm.random() / 100
		dec_lon = rm.random() / 100
		data.append((hex1.lower(), flt, lon + dec_lon, lat + dec_lat))
	if save_file_name:
		df = pd.DataFrame(data)
		df.to_csv(save_file_name, header = ['hex1', 'flt', 'dec_lat', 'dec_lon'], index = False)
	return data

def sacar_curp(nombre, apellido_paterno, apellido_materno, fecha_nac, sexo, estado_nac):
	fechanac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += dt.datetime.strftime(fechanac, '%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS',
		'baja_california': 'BC',
		'baja_california_sur': 'BS',
		'campeche': 'CC',
		'coahuila': 'CL',
		'colima': 'CM',
		'chiapas': 'CS',
		'chihuahua': 'CH',
		'distrito_federal': 'DF',
		'durango': 'DG',
		'guanajuato': 'GT',
		'guerrero': 'GR',
		'hidalgo': 'HG',
		'jalisco': 'JC',
		'mexico': 'MC',
		'michoacan': 'MN',
		'morelos': 'MS',
		'nayarit': 'NT',
		'nuevo_leon': 'NL',
		'oaxaca': 'OC',
		'puebla': 'PL',
		'queretaro': 'QT',
		'quintana_roo': 'QR',
		'san_luis_potosi': 'SP',
		'sinaloa': 'SL',
		'sonora': 'SR',
		'tabasco': 'TC',
		'tamaulipas': 'TS',
		'tlaxcala': 'TL',
		'veracruz': 'VZ',
		'yucatan': 'YN',
		'zacatecas': 'ZS'
	}
	clave_estado_nac = clave_estados[estado_nac.replace(' ', '_').lower()]
	curp += clave_estado_nac
	# Primera consonante interna del primer apellido
	pcap = re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	curp += pcap if pcap != 'Ñ' else 'X'
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fechanac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

In [89]:
def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for _ in range(n):
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		margen_ap = reap[(reap['edad'] == edad) if edad <= reap['edad'].max() else (reap['edad'] == reap['edad'].max())].reset_index()

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'][0], margen_ap['lim_sup.altura.h.cm'][0]), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'][0], margen_ap['lim_sup.peso.h.kg'][0]), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'][0], margen_ap['lim_sup.altura.m.cm'][0]), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'][0], margen_ap['lim_sup.peso.m.kg'][0]), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)

		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		id_estado = rm.choice(ciudades['id_estado'].unique())
		ct_ciudad = ciudades[ciudades['id_estado'] == id_estado].sample()

		ciudad = ct_ciudad['ciudad'].values[0]
		estado = ct_ciudad['estado'].values[0]
		tel = f'({ct_ciudad["clave_lada"].values[0]})' + str(rm.randint(1000000, 9999999))
		cel = f'({ct_ciudad["clave_lada"].values[0]})' + str(rm.randint(1000000, 9999999))

		direccion = fake.street_address()

		direcc_calle = fake.street_name()
		direcc_numero = rm.randint(1, 999)
		direcc_cp = fake.postcode()

		profesion = profesiones.sample().values[0][0] if edad >= 18 else None
		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos en dicha institución.
		El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		if profesion:
			# Los primeros dos caracteres que lo componen están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			nss2 = ('%02d' % rm.randint(0, 99))
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%02d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'
		else:
			nss = None

		"""
		Latitud, Longitud de la ciudad
		"""
		# si el catalogo ya tiene coordenadas puestas, tomar esos valores
		if pd.notna(ct_ciudad['latitud'].values[0]) and pd.notna(ct_ciudad['longitud'].values[0]):
			lat = ct_ciudad['latitud'].values[0]
			lon = ct_ciudad['longitud'].values[0]
		else:
			# si no, buscar en internet la latitud y longitud segun la ciudad
			try:
				# Requiere internet
				geolocator = Nominatim(user_agent = 'geoapiExercises')
				location = geolocator.geocode(f'{ciudad}, {estado}, Mexico', language = 'es-MX')
				if location:
					lat = location.latitude
					lon = location.longitude

					# for _ in range(5):
					# 	dec_lat = rm.random() / 100
					# 	dec_lon = rm.random() / 100
					# 	dirr = geolocator.reverse((lat + dec_lat, lon + dec_lon), language = 'es', addressdetails = True)
					# 	print(dirr)

					# guardar coordenadas en el catalogo
					# para que a la otra no haya que buscar en internet
					ct_ciudad_index_row = ct_ciudad.index.values[0]
					ciudades.loc[ct_ciudad_index_row, 'latitud'] = lat
					ciudades.loc[ct_ciudad_index_row, 'longitud'] = lon
					ciudades.to_csv('./catalogos/ciudades.csv', index = False)
				else:
					raise BaseException(f'(idx {_}) No se encontro latitud ni longitud de {ciudad}, {estado}')
			except BaseException as be:
				# En caso de no haber encontrado o no tener internet
				print('\033[31mException:\033[0m')
				print(be)
				lat = lon = np.NaN

		"""
		Zona horaria
		"""
		if not np.isnan(lat) and not np.isnan(lon):
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			timezone = obj.timezone_at(lat = lat, lng = lon)
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')
			# en Enero no hay DST
			#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
		else:
			timezone = tz_offset = None

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'correo': correo,
			'tel': tel,
			# 'calle': '',
			# 'numero': '',
			# 'cp': '',
			'ciudad': ciudad,
			'estado': estado,
			'lat': lat,
			'lon': lon,
			'zona_horaria': timezone,
			'tz_offset': tz_offset,
			'profesion': profesion,
			'nss': nss,
			'color_fav': fake.safe_color_name(),
		})
	return pd.DataFrame(personas)

personas_aleatorias(10)

,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,correo,tel,ciudad,estado,lat,lon,zona_horaria,tz_offset,profesion,nss,color_fav
0,None,Ramses,Angulo,Prado,2000-08-12,H,177.61,64.77,B+,ramsesanguloprado@example.com,(597)7776346,Amecameca,Estado de Mexico,19.120025,-98.706532,America/Mexico_City,-0600,Gerente del servicio al cliente,57 74 00 6757 5,fuchsia
1,None,Alejandro,Grijalva,Urrutia,2014-09-09,H,119.72,26.79,A+,agu14@example.gob,(449)4657930,General Ignacio Zaragoza,Aguascalientes,21.900753,-102.299178,America/Mexico_City,-0600,None,None,white
2,Sra.,Alicia,Escalante,Canales,1964-11-25,M,155.03,82.19,A+,aec64@example.org,(672)2573236,Altata,Sinaloa,24.634138,-107.930019,America/Mazatlan,-0700,Ejecutivo de cuentas de publicidad,89 92 64 65 7,teal
3,None,Ernesto,Longoria,Sosa,1942-07-25,H,181.31,83.58,A-,ernestolongoriasosa@example.io,(241)5689474,Apizaco,Tlaxcala,19.426431,-98.077433,America/Mexico_City,-0600,Trabajador de desarrollo comunitario,30 45 42 2775 5,teal
4,None,Marisela,Gallegos,Guerrero,2008-07-26,M,166.36,61.78,O-,mariselagallegosguerrero@example.org,(33)4855893,Palomar,Jalisco,20.591484,-103.448769,America/Mexico_City,-0600,None,None,yellow
5,None,Aldonza,Guardado,Godoy,1964-03-06,M,162.98,77.59,O+,agg1964@example.gob,(751)6815760,Huajintlán,Morelos,18.609664,-99.425394,America/Mexico_City,-0600,Operador de cámara,77 97 64 8516 5,yellow
6,None,Eloisa,Cuellar,del Valle,1935-11-16,M,166.27,75.41,A+,uanaya@example.com,(55)5624941,San Miguel Topilejo,Ciudad de Mexico,19.201305,-99.141667,America/Mexico_City,-0600,Tesorero corporativo,32 59 35 1059 1,lime
7,None,Rubén,Peres,Zapata,1980-12-15,H,172.03,70.03,O+,lilia54@example.io,(832)5858732,Llera de Canales,Tamaulipas,23.318088,-99.025813,America/Monterrey,-0600,Gerente de galería de arte comercial,71 18 80 4049 1,fuchsia
8,None,Fabiola,Leyva,Cepeda,2003-04-01,M,157.39,79.25,A+,fleyva@example.org,(483)9937045,Matlapa,San Luis Potosí,21.332648,-98.818830,America/Mexico_City,-0600,Gerente de galería de arte,73 99 03 8474 3,maroon
9,None,Cristina,Delgadillo,Paz,1982-06-30,M,171.25,45.74,A+,cdp82@example.edu,(951)5676536,San Dionisio Ocotepec,Oaxaca,16.785197,-96.306086,America/Mexico_City,-0600,Tripulación de cabina aérea,46 15 82 9348 0,olive


In [117]:
# mas info en https://randomuser.me/documentation
url_api = 'https://randomuser.me/api?format=csv'
params = {
	'results': 100,
	#'nat': 'es',
	#'password': '8-16,lower,upper,number,special',
	'exc': 'nat,picture,id,login,registered'
}
query = ''
for k, v in params.items():
	if isinstance(v, int): v = str(v)
	query += '&' + k + '=' + v
url = url_api + query
people = pd.read_csv(url)
people.sample(8)

,gender,name.title,name.first,name.last,location.street.number,location.street.name,location.city,location.state,location.country,location.postcode,location.coordinates.latitude,location.coordinates.longitude,location.timezone.offset,location.timezone.description,email,dob.date,dob.age,phone,cell
62,male,Mr,Nicklas,Johansen,1217,Frederiksborgvej,Nørrebro,Syddanmark,Denmark,27372,33.3072,157.7187,+4:30,Kabul,nicklas.johansen@example.com,1965-09-14T23:16:38.308Z,57,77401924,91068794
60,female,Miss,Embla,Skjæret,1537,Eikeveien,Levanger,Buskerud,Norway,4642,22.6395,116.0633,-1:00,"Azores, Cape Verde Islands",embla.skjaeret@example.com,1973-10-08T09:58:07.762Z,49,68550908,45269636
25,male,Mr,Eli,Singh,1882,Gladstone Road,Greymouth,Waikato,New Zealand,79550,77.6386,151.6528,-5:00,"Eastern Time (US & Canada), Bogota, Lima",eli.singh@example.com,1971-04-15T11:37:58.102Z,51,(547)-178-5107,(026)-115-1175
85,female,Mrs,Freja,Madsen,2714,Strandvej,Randers Nv,Syddanmark,Denmark,26549,-20.2449,-171.2555,+9:00,"Tokyo, Seoul, Osaka, Sapporo, Yakutsk",freja.madsen@example.com,1966-09-29T02:28:48.384Z,56,27780652,28807534
80,male,Mr,Jar,Kelley,7197,Brown Terrace,Killeen,Alaska,United States,41584,-0.2664,-67.0959,0:00,"Western Europe Time, London, Lisbon, Casablanca",jar.kelley@example.com,1968-03-22T05:11:36.035Z,54,(775)-963-0793,(294)-398-8814
48,male,Mr,Xavier,Jones,7120,Pine Rd,Georgetown,Saskatchewan,Canada,Q1O 7V7,66.1318,40.9410,+6:00,"Almaty, Dhaka, Colombo",xavier.jones@example.com,1996-02-11T08:13:20.101Z,26,664-304-9491,520-836-3330
26,female,Mademoiselle,Mia,Leroy,2148,Rue Baraban,Oulens-Sous-Echallens,Thurgau,Switzerland,3884,42.3597,177.8986,+6:00,"Almaty, Dhaka, Colombo",mia.leroy@example.com,1996-08-03T22:44:45.110Z,26,076 331 29 17,078 480 88 17
36,male,Mr,Mehmet,Ayaydın,8779,Atatürk Sk,Kastamonu,Muğla,Turkey,62397,10.4526,-56.7678,-7:00,Mountain Time (US & Canada),mehmet.ayaydin@example.com,1945-07-30T13:17:07.251Z,77,(653)-673-2668,(491)-149-0570
